# Lotto XGBoost with 5-Number Buckets (Named Ranges)

This notebook:

1. Loads `Lotto_DATE.xlsx` (NZ Lotto draw history).
2. Builds **5-number bucket features** with meaningful names, e.g. `bucket_1_5_count`, `bucket_6_10_count`, etc.
3. Adds simple extra features (Odd/Even, date parts, etc.).
4. Trains an **XGBoost multi-output regressor** (one target per winning number).
5. Performs 5-fold cross-validation.
6. Demonstrates generating candidate draws and predicting with the best model.
7. Saves the expanded dataset (with new features) as `Lotto5_Imputed.xlsx`.

Place this notebook in the same folder as `Lotto5.xlsx` and run all cells top to bottom.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.multioutput import MultiOutputRegressor

from xgboost import XGBRegressor

np.random.seed(42)
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', 100)

print('Libraries imported.')

Libraries imported.


In [2]:
# Path to your Excel file. Ensure Lotto_2025-11-26.xlsx is in the same directory as this notebook.
excel_path = "Lotto_2025-11-26.xlsx"

df = pd.read_excel(excel_path)
print("Data shape:", df.shape)
display(df.head())
print("\nColumns:", list(df.columns))

Data shape: (1464, 32)


,Draw,Date,Winning Number 1,2,3,4,5,6,Supplementary Number,From Last,Low,High,Odd,Even,1-10,11-20,21-30,31-40,Division 1 Winners,Division 1 Prize,Division 2 Winners,Division 2 Prize,Division 3 Winners,Division 3 Prize,Division 4 Winners,Division 4 Prize,Division 5 Winners,Division 5 Prize,Division 6 Winners,Division 6 Prize,Division 7 Winners,Division 7 Prize
0,2537,2025-11-26,4,5,9,21,32,34,3,NaN,3,3,3,3,3,0,1,2,2.0,500000.0,13.0,16290.0,300.0,700.0,802.0,60.0,12979.0,32.0,18258.0,22.0,185398.0,1.5
1,2536,2025-11-22,2,6,10,12,28,30,22,"2,10",4,2,0,6,3,1,2,0,5.0,200000.0,22.0,11489.0,548.0,457.0,1372.0,42.0,20291.0,25.0,26709.0,18.0,254719.0,1.5
2,2535,2025-11-19,2,10,18,25,29,31,21,2,3,3,3,3,2,1,2,1,0.0,0.0,15.0,16772.0,423.0,590.0,1088.0,53.0,17552.0,28.0,22725.0,21.0,232598.0,1.5
3,2534,2025-11-15,2,5,14,19,28,38,29,19,4,2,2,4,2,2,1,1,18.0,55556.0,62.0,16909.0,1777.0,585.0,4196.0,57.0,68364.0,30.0,89904.0,23.0,952797.0,1.5
4,2533,2025-11-12,4,10,18,19,27,30,32,4,4,2,2,4,2,2,2,0,6.0,166667.0,20.0,32541.0,1172.0,551.0,2489.0,59.0,46570.0,28.0,56635.0,22.0,626052.0,1.5



Columns: ['Draw', 'Date', 'Winning Number 1', '2', '3', '4', '5', '6', 'Supplementary Number', 'From Last', 'Low', 'High', 'Odd', 'Even', '1-10', '11-20', '21-30', '31-40', 'Division 1 Winners', 'Division 1 Prize', 'Division 2 Winners', 'Division 2 Prize', 'Division 3 Winners', 'Division 3 Prize', 'Division 4 Winners', 'Division 4 Prize', 'Division 5 Winners', 'Division 5 Prize', 'Division 6 Winners', 'Division 6 Prize', 'Division 7 Winners', 'Division 7 Prize']


## Basic cleaning and helper columns

We:

- Ensure the winning number columns are numeric.
- Convert helper columns like `From Last`, `Same As Day`, `Odd`, `Even` to numeric (if present).
- Parse `Date` and add simple date features.


In [3]:
# Winning number columns (targets)
number_cols = [
    "Winning Number 1",
    "Winning Number 2",
    "Winning Number 3",
    "Winning Number 4",
    "Winning Number 5",
    "Winning Number 6",
]

# Ensure numeric for winning numbers
for col in number_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    else:
        raise ValueError(f"Expected column '{col}' not found in dataframe.")

# Helper columns that may exist
helper_cols = ["From Last", "Same As Day", "Odd", "Even"]
for col in helper_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Date handling
if "Date" in df.columns:
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df["Year"] = df["Date"].dt.year.fillna(0).astype(int)
    df["Month"] = df["Date"].dt.month.fillna(0).astype(int)
    df["DayOfWeek"] = df["Date"].dt.dayofweek.fillna(0).astype(int)
else:
    df["Year"] = 0
    df["Month"] = 0
    df["DayOfWeek"] = 0

print("After basic cleaning:")
display(df.head())

ValueError: Expected column 'Winning Number 2' not found in dataframe.

## 5-number bucket features with named ranges

We map numbers as follows (for NZ Lotto 1–40):

- 1–5   → bucket index 0 → features: `bucket_1_5_count`, `bucket_1_5_present`
- 6–10  → bucket index 1 → `bucket_6_10_count`, ...
- 11–15 → bucket index 2
- 16–20 → bucket index 3
- 21–25 → bucket index 4
- 26–30 → bucket index 5
- 31–35 → bucket index 6
- 36–40 → bucket index 7

So you can immediately see which 5-number range each feature refers to.


In [ ]:
def num_to_bucket(num: float, bucket_size: int = 5) -> float:
    """Map a lotto number to a 0-based bucket index of size `bucket_size`.
    Returns NaN for missing values.
    """
    if pd.isna(num):
        return np.nan
    return int((int(num) - 1) // bucket_size)

# Create per-number bucket columns (indices)
for col in number_cols:
    df[f"{col}_bucket"] = df[col].apply(num_to_bucket)

bucket_cols = [f"{col}_bucket" for col in number_cols]
max_bucket = int(df[bucket_cols].max().max())

# Infer max actual number from data (e.g. 40)
max_number = int(df[number_cols].max().max())
bucket_size = 5

# Build mapping from bucket index -> human-readable column names
bucket_index_to_count_col = {}
bucket_index_to_present_col = {}
bucket_count_cols = []
bucket_present_cols = []

for i in range(max_bucket + 1):
    low = i * bucket_size + 1
    high = min((i + 1) * bucket_size, max_number)
    count_name = f"bucket_{low}_{high}_count"
    present_name = f"bucket_{low}_{high}_present"
    bucket_index_to_count_col[i] = count_name
    bucket_index_to_present_col[i] = present_name
    bucket_count_cols.append(count_name)
    bucket_present_cols.append(present_name)

print("Bucket index to feature names:")
for i in range(max_bucket + 1):
    print(i, "->", bucket_index_to_count_col[i], ",", bucket_index_to_present_col[i])

display(df[bucket_cols].head())

In [ ]:
# Compute bucket counts per draw using the named columns
def bucket_count_row(row):
    counts = np.zeros(max_bucket + 1, dtype=int)
    buckets = row[bucket_cols].values
    for b in buckets:
        if not pd.isna(b):
            b_int = int(b)
            if 0 <= b_int <= max_bucket:
                counts[b_int] += 1
    # Map counts into named columns
    data = {}
    for i in range(max_bucket + 1):
        data[bucket_index_to_count_col[i]] = counts[i]
    return pd.Series(data, index=bucket_count_cols)

df_bucket_counts = df.apply(bucket_count_row, axis=1)
df = pd.concat([df, df_bucket_counts], axis=1)

# Presence flags using named columns
for i in range(max_bucket + 1):
    count_col = bucket_index_to_count_col[i]
    present_col = bucket_index_to_present_col[i]
    df[present_col] = (df[count_col] > 0).astype(int)

# Bucket energy (weighted sum of bucket indices by count)
df["bucket_energy"] = 0
for i in range(max_bucket + 1):
    count_col = bucket_index_to_count_col[i]
    df["bucket_energy"] += i * df[count_col]

print("Bucket features created (named ranges):")
display(df.head())

In [ ]:
# Save expanded dataset with new features
output_excel_path = "Lotto_2025-11-26_Imputed.xlsx"
df.to_excel(output_excel_path, index=False)
print(f"Saved dataset with new features to {output_excel_path}")

## Build feature matrix and target matrix

- **Targets**: the six winning numbers as a 6D regression target.
- **Features**: named bucket counts/presence, bucket energy, helper columns, and date parts.


In [ ]:
target_cols = number_cols.copy()

# bucket_count_cols and bucket_present_cols already defined with meaningful names
candidate_feature_cols = (
    bucket_count_cols
    + bucket_present_cols
    + ["bucket_energy", "From Last", "Same As Day", "Odd", "Even", "Year", "Month", "DayOfWeek"]
)

# Keep only columns that exist in df (in case some helper cols are missing)
feature_cols = [c for c in candidate_feature_cols if c in df.columns]

print("Using feature columns:")
print(feature_cols)

X = df[feature_cols].values
y = df[target_cols].values

print("Feature matrix shape:", X.shape)
print("Target matrix shape:", y.shape)

## XGBoost model with cross-validation

We use:

- `SimpleImputer` to handle any missing feature values.
- `MultiOutputRegressor(XGBRegressor)` to predict all 6 numbers at once.
- 5-fold cross-validation with **negative MSE** scoring.


In [ ]:
# Define base XGBoost regressor
xgb_reg = XGBRegressor(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    tree_method="hist",  # change to 'gpu_hist' if you have GPU XGBoost installed
    random_state=42,
)

model = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("regressor", MultiOutputRegressor(xgb_reg)),
    ]
)

scoring = make_scorer(mean_squared_error, greater_is_better=False)

cv_results = cross_validate(
    model,
    X,
    y,
    cv=5,
    scoring=scoring,
    return_estimator=True,
    n_jobs=-1,
)

test_scores = -cv_results["test_score"]  # convert back to positive MSE
print("Cross-validation MSE scores:", test_scores)
print("Mean CV MSE:", np.mean(test_scores))

best_model_index = np.argmin(test_scores)
best_model = cv_results["estimator"][best_model_index]
print("Best model index:", best_model_index)

## Predicting from candidate draws

To keep things simple, we:

1. Generate a **candidate draw** (6 random numbers 1–40, no replacement).
2. Build the same **bucket-based features** (with named ranges) for that draw.
3. Use the best cross-validated model to predict a 6D output.
4. Map predictions back into the 1–40 range (wrapping with modulo).

This is more for exploration / "pattern resonance" than real prediction.


In [ ]:
def build_features_from_draw(draw_numbers, feature_columns, max_bucket_local=None, bucket_size_local=5):
    """Build a one-row feature DataFrame for a candidate draw using the same
    bucket logic and feature columns as the training data.
    """
    draw_numbers = np.array(draw_numbers, dtype=int)
    if max_bucket_local is None:
        max_bucket_local = max_bucket

    # bucket counts
    counts = np.zeros(max_bucket_local + 1, dtype=int)
    for n in draw_numbers:
        b = num_to_bucket(n, bucket_size=bucket_size_local)
        if 0 <= b <= max_bucket_local:
            counts[b] += 1

    row = {}

    # bucket counts and presence using named columns
    for i in range(max_bucket_local + 1):
        count_col = bucket_index_to_count_col[i]
        present_col = bucket_index_to_present_col[i]
        if count_col in feature_columns:
            row[count_col] = counts[i]
        if present_col in feature_columns:
            row[present_col] = int(counts[i] > 0)

    # bucket_energy
    if "bucket_energy" in feature_columns:
        row["bucket_energy"] = sum(i * counts[i] for i in range(max_bucket_local + 1))

    # draw-level stats (only if these columns exist in the model)
    if "draw_median" in feature_columns:
        row["draw_median"] = float(np.median(draw_numbers))
    if "draw_mean" in feature_columns:
        row["draw_mean"] = float(np.mean(draw_numbers))
    if "draw_min" in feature_columns:
        row["draw_min"] = int(np.min(draw_numbers))
    if "draw_max" in feature_columns:
        row["draw_max"] = int(np.max(draw_numbers))

    # Odd / Even counts (if used as features)
    if "Odd" in feature_columns:
        row["Odd"] = int((draw_numbers % 2 != 0).sum())
    if "Even" in feature_columns:
        row["Even"] = int((draw_numbers % 2 == 0).sum())

    # helper + date features (neutral defaults if present in training)
    defaults = {
        "From Last": 0,
        "Same As Day": 0,
        "Year": 0,
        "Month": 0,
        "DayOfWeek": 0,
    }
    for col, val in defaults.items():
        if col in feature_columns and col not in row:
            row[col] = val

    # ensure all feature_columns exist
    for col in feature_columns:
        if col not in row:
            row[col] = 0

    return pd.DataFrame([row], columns=feature_columns)


def sanitize_prediction(raw_pred, n_numbers=6, low=1, high=40):
    """Convert raw model outputs into a valid NZ Lotto line:
    - Round to ints
    - Wrap into [low, high] range
    - Enforce uniqueness (no duplicates)
    - Return sorted array of length n_numbers
    """
    # Map floats to ints in the valid range
    ints = ((np.round(raw_pred).astype(int) - 1) % high) + low
    ints = np.clip(ints, low, high)

    # Enforce uniqueness
    uniq = np.unique(ints)

    # If we have fewer than required, sample extra distinct numbers
    if len(uniq) < n_numbers:
        remaining = np.setdiff1d(np.arange(low, high + 1), uniq)
        extra = np.random.choice(remaining, size=n_numbers - len(uniq), replace=False)
        uniq = np.sort(np.concatenate([uniq, extra]))
    # If we have more, truncate to the first n_numbers
    elif len(uniq) > n_numbers:
        uniq = np.sort(uniq)[:n_numbers]
    else:
        uniq = np.sort(uniq)

    return uniq


# Example: generate a few candidate draws and predict
num_predictions = 5
for i in range(num_predictions):
    candidate_numbers = np.sort(np.random.choice(np.arange(1, 41), size=6, replace=False))
    input_df = build_features_from_draw(candidate_numbers, feature_cols)

    # Prefer the Monte Carlo-augmented model if available
    if "model_mc" in globals():
        pred = model_mc.predict(input_df.values)[0]
        model_used = "model_mc (augmented)"
    else:
        pred = best_model.predict(input_df.values)[0]
        model_used = "best_model (original CV best)"

    predicted_numbers = sanitize_prediction(pred, n_numbers=6, low=1, high=40)

    print(f"Prediction set {i+1} using {model_used}:")
    print("  Candidate base numbers:", candidate_numbers)
    print("  Model predicted numbers:", predicted_numbers)
    print("-" * 60)

## Monte Carlo Lotto Simulator, Bucket Energy & Delta Export

This section:

- Reuses the existing 5-number bucket logic.
- Computes additional draw-level features like `draw_median`, `draw_mean`, `draw_min`, `draw_max`.
- Visualises **bucket energy** and **median** over time.
- Builds an **empirical + uniform blended Monte Carlo simulator** to generate many synthetic draws.
- Computes the same bucket features for synthetic draws.
- Optionally saves both real and synthetic feature tables to **Delta Lake** for ROAPI or other tools.


In [ ]:
# Use the existing dataframe `df` and `number_cols` from earlier cells.
# If your raw draws dataframe has a different name, adjust `df_draws` accordingly.
df_draws = df  # alias for clarity

number_cols = number_cols  # ensure we use the same target columns

print("Using number columns:", number_cols)

def compute_bucket_features_for_draws(df_base: pd.DataFrame) -> pd.DataFrame:
    """Compute bucket count/presence, bucket_energy and draw-level stats for each draw."""
    df_feat = df_base.copy()

    # Map each number to a bucket index using existing num_to_bucket
    bucket_indices = df_feat[number_cols].applymap(num_to_bucket)

    # Count per bucket into the named bucket_*_count columns
    def row_to_bucket_counts(row):
        counts = np.zeros(max_bucket + 1, dtype=int)
        for b in row.values:
            if not pd.isna(b):
                b_int = int(b)
                if 0 <= b_int <= max_bucket:
                    counts[b_int] += 1
        data = {}
        for i in range(max_bucket + 1):
            data[bucket_index_to_count_col[i]] = counts[i]
        return pd.Series(data, index=bucket_count_cols)

    df_counts = bucket_indices.apply(row_to_bucket_counts, axis=1)
    df_feat[df_counts.columns] = df_counts

    # Presence flags using named columns
    for i in range(max_bucket + 1):
        count_col = bucket_index_to_count_col[i]
        present_col = bucket_index_to_present_col[i]
        df_feat[present_col] = (df_feat[count_col] > 0).astype(int)

    # Bucket energy: sum(bucket_index * count)
    df_feat["bucket_energy"] = df_counts.apply(
        lambda row: sum(i * row[bucket_index_to_count_col[i]] for i in range(max_bucket + 1)),
        axis=1,
    )

    # Draw-level stats from the actual numbers
    df_feat["draw_median"] = df_feat[number_cols].median(axis=1)
    df_feat["draw_mean"] = df_feat[number_cols].mean(axis=1)
    df_feat["draw_min"] = df_feat[number_cols].min(axis=1)
    df_feat["draw_max"] = df_feat[number_cols].max(axis=1)

    # Odd / Even counts
    df_feat["Odd"] = df_feat[number_cols].apply(lambda r: (r % 2 != 0).sum(), axis=1)
    df_feat["Even"] = df_feat[number_cols].apply(lambda r: (r % 2 == 0).sum(), axis=1)

    return df_feat

df_real_feat = compute_bucket_features_for_draws(df_draws)
print("Real feature table shape:", df_real_feat.shape)
df_real_feat.head()

In [ ]:
# Visualisations: bucket_energy & draw_median over time, plus distributions
x_axis = df_real_feat["Date"] if "Date" in df_real_feat.columns else df_real_feat.index

plt.figure(figsize=(10, 4))
plt.plot(x_axis, df_real_feat["bucket_energy"])
plt.title("Bucket Energy Over Time (Real Draws)")
plt.xlabel("Draw")
plt.ylabel("Bucket Energy")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.plot(x_axis, df_real_feat["draw_median"])
plt.title("Draw Median Over Time (Real Draws)")
plt.xlabel("Draw")
plt.ylabel("Median of Drawn Numbers")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 4))
plt.hist(df_real_feat["bucket_energy"], bins=20)
plt.title("Distribution of Bucket Energy (Real Draws)")
plt.xlabel("Bucket Energy")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

avg_counts = df_real_feat[bucket_count_cols].mean()
plt.figure(figsize=(10, 3))
plt.bar(range(len(avg_counts)), avg_counts.values)
plt.xticks(range(len(avg_counts)), avg_counts.index, rotation=45)
plt.title("Average Count per 5-Number Bucket (Real Draws)")
plt.ylabel("Avg Count per Draw")
plt.tight_layout()
plt.show()

In [ ]:
# Empirical distribution of numbers from real draws
def compute_empirical_probs(df_base: pd.DataFrame, number_columns) -> pd.Series:
    nums = df_base[number_columns].values.ravel()
    values, counts = np.unique(nums, return_counts=True)
    n_min = int(nums.min())
    n_max = int(nums.max())
    all_numbers = np.arange(n_min, n_max + 1)

    freq = pd.Series(0, index=all_numbers, dtype=float)
    freq.loc[values] = counts
    p_emp = freq / freq.sum()
    return p_emp

p_empirical = compute_empirical_probs(df_draws, number_cols)
p_uniform = pd.Series(1.0 / len(p_empirical), index=p_empirical.index, dtype=float)

# Blend empirical with uniform so we expand the observed pattern distribution
alpha = 0.7  # 70% empirical, 30% uniform
p_blended = alpha * p_empirical + (1 - alpha) * p_uniform
p_blended = p_blended / p_blended.sum()

print("First few blended probabilities:")
print(p_blended.head())

def simulate_single_draw(draw_size=6, probs: pd.Series = p_blended):
    """Simulate one lotto draw by sampling without replacement from the blended distribution."""
    numbers = probs.index.to_numpy()
    p = probs.values
    sample = np.random.choice(numbers, size=draw_size, replace=False, p=p)
    return np.sort(sample)

def simulate_dataset(n_draws=100_000, draw_size=6, probs: pd.Series = p_blended):
    draws = [simulate_single_draw(draw_size, probs) for _ in range(n_draws)]
    df_sim = pd.DataFrame(draws, columns=number_cols[:draw_size])
    df_sim["SimDrawID"] = np.arange(1, n_draws + 1)
    # Synthetic dates purely for plotting/ordering
    df_sim["SimDate"] = pd.date_range("2000-01-01", periods=n_draws, freq="D")
    return df_sim

N_SYNTH_DRAWS = 50_000  # adjust as you like
df_synth = simulate_dataset(N_SYNTH_DRAWS)
print("Synthetic draws shape:", df_synth.shape)
df_synth.head()

In [ ]:
# Compute the same bucket features for synthetic draws
df_synth_feat = compute_bucket_features_for_draws(df_synth)

print("Average bucket counts (real):")
print(df_real_feat[bucket_count_cols].mean().round(3))

print("\nAverage bucket counts (synthetic):")
print(df_synth_feat[bucket_count_cols].mean().round(3))

# Optional: synthetic bucket energy distribution
plt.figure(figsize=(6, 4))
plt.hist(df_synth_feat["bucket_energy"], bins=20)
plt.title("Distribution of Bucket Energy (Synthetic Draws)")
plt.xlabel("Bucket Energy")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

## Delta Export, Monte Carlo-Augmented Training, and Lotto Number Generation

In [ ]:
from pathlib import Path

# Delta Lake export for real + synthetic feature tables
try:
    from deltalake import write_deltalake
    import pyarrow as pa
    HAS_DELTALAKE = True
except ImportError:
    HAS_DELTALAKE = False
    print("Warning: 'deltalake' or 'pyarrow' not installed; skipping Delta writes.")
    print("Install with: pip install deltalake pyarrow")
    
delta_base = Path("delta_lotto")
delta_base.mkdir(parents=True, exist_ok=True)

if HAS_DELTALAKE:
    real_delta_path = delta_base / "real_features"
    synth_delta_path = delta_base / "synthetic_features"

    # Convert pandas DataFrames to Arrow tables explicitly
    real_table = pa.Table.from_pandas(df_real_feat, preserve_index=False)
    synth_table = pa.Table.from_pandas(df_synth_feat, preserve_index=False)

    print(f"Writing real feature table to Delta: {real_delta_path}")
    write_deltalake(str(real_delta_path), real_table, mode="overwrite")

    print(f"Writing synthetic feature table to Delta: {synth_delta_path}")
    write_deltalake(str(synth_delta_path), synth_table, mode="overwrite")

    print("Delta export complete.")
else:
    print("Delta export skipped.")

In [ ]:
# Retrain XGBoost on augmented dataset: real + synthetic Monte Carlo draws
# We reuse:
# - feature_cols: feature column list used originally
# - target_cols: the six winning-number columns
# - model: the Pipeline(imputer + MultiOutputRegressor(XGBRegressor))

# Ensure we have the target columns present in both real and synthetic feature tables
missing_targets_real = [c for c in target_cols if c not in df_real_feat.columns]
missing_targets_synth = [c for c in target_cols if c not in df_synth_feat.columns]

if missing_targets_real or missing_targets_synth:
    print("Warning: some target columns are missing in real or synthetic tables.")
    print("Missing in real:", missing_targets_real)
    print("Missing in synthetic:", missing_targets_synth)
else:
    # Build augmented dataframe with just the columns we need
    cols_needed = feature_cols + target_cols

    # Ensure synthetic feature table has all required columns; fill missing with 0
    for col in cols_needed:
        if col not in df_synth_feat.columns:
            df_synth_feat[col] = 0

    # Also ensure real feature table has all required columns (defensive)
    for col in cols_needed:
        if col not in df_real_feat.columns:
            df_real_feat[col] = 0

    df_real_aug = df_real_feat[cols_needed].copy()
    df_synth_aug = df_synth_feat[cols_needed].copy()

    df_aug = pd.concat([df_real_aug, df_synth_aug], axis=0, ignore_index=True)
    print("Augmented dataset shape:", df_aug.shape)

    X_aug = df_aug[feature_cols].values
    y_aug = df_aug[target_cols].values

    print("Augmented feature matrix shape:", X_aug.shape)
    print("Augmented target matrix shape:", y_aug.shape)

    # Fit a new model on the full augmented dataset (no extra CV here)
    model_mc = model.fit(X_aug, y_aug)
    print("Monte Carlo-augmented XGBoost model trained and stored as 'model_mc'.")

In [ ]:
# Generate example Lotto predictions using the current model (prefers model_mc if available)
num_predictions = 5
for i in range(num_predictions):
    candidate_numbers = np.sort(np.random.choice(np.arange(1, 41), size=6, replace=False))
    input_df = build_features_from_draw(candidate_numbers, feature_cols)

    if "model_mc" in globals():
        pred = model_mc.predict(input_df.values)[0]
        model_used = "model_mc (augmented)"
    else:
        pred = best_model.predict(input_df.values)[0]
        model_used = "best_model (original CV best)"

    predicted_numbers = sanitize_prediction(pred, n_numbers=6, low=1, high=40)

    print(f"Prediction set {i+1} using {model_used}:")
    print("  Candidate base numbers:", candidate_numbers)
    print("  Model predicted numbers:", predicted_numbers)
    print("-" * 60)

## Pattern Resonance Scoring & Extra Visualisations

This section adds:
- A **pattern resonance score** for any predicted Lotto line, comparing it to historical draw structure.
- Extra plots comparing **real vs synthetic** distributions and highlighting where model-generated lines sit in feature space.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Pattern resonance helpers ---

def line_to_features(line_numbers, max_bucket_local=None, bucket_size_local=5):
    """Compute a minimal feature dict for a single line of numbers:
    - bucket counts
    - bucket_energy
    - draw_median
    - Odd / Even counts
    """
    line_numbers = np.array(line_numbers, dtype=int)
    if max_bucket_local is None:
        max_bucket_local = max_bucket

    # bucket counts
    counts = np.zeros(max_bucket_local + 1, dtype=int)
    for n in line_numbers:
        b = num_to_bucket(n, bucket_size=bucket_size_local)
        if 0 <= b <= max_bucket_local:
            counts[b] += 1

    feats = {}
    feats["bucket_counts"] = counts
    feats["bucket_energy"] = sum(i * counts[i] for i in range(max_bucket_local + 1))
    feats["draw_median"] = float(np.median(line_numbers))
    feats["Odd"] = int((line_numbers % 2 != 0).sum())
    feats["Even"] = int((line_numbers % 2 == 0).sum())
    return feats


def compute_historical_profiles(df_real_feat, bucket_count_cols):
    """Precompute historical reference statistics from real draws."""
    hist = {}
    # Average bucket counts per draw
    hist["mean_bucket_counts"] = df_real_feat[bucket_count_cols].mean().values
    hist["std_bucket_counts"] = df_real_feat[bucket_count_cols].std(ddof=0).values

    # bucket_energy stats
    hist["mean_bucket_energy"] = df_real_feat["bucket_energy"].mean()
    hist["std_bucket_energy"] = df_real_feat["bucket_energy"].std(ddof=0)

    # draw_median stats (if present)
    if "draw_median" in df_real_feat.columns:
        hist["mean_draw_median"] = df_real_feat["draw_median"].mean()
        hist["std_draw_median"] = df_real_feat["draw_median"].std(ddof=0)
    else:
        hist["mean_draw_median"] = None
        hist["std_draw_median"] = None

    # Odd/Even stats if available
    if "Odd" in df_real_feat.columns and "Even" in df_real_feat.columns:
        hist["mean_odd"] = df_real_feat["Odd"].mean()
        hist["mean_even"] = df_real_feat["Even"].mean()
    else:
        hist["mean_odd"] = None
        hist["mean_even"] = None

    return hist


def pattern_resonance_score(line_numbers, hist_profile, bucket_count_cols):
    """Compute a 0–1 'pattern resonance' score indicating how Lotto-like
    a given line is, compared to historical patterns.

    Uses:
    - Bucket count distance
    - Bucket energy distance (z-score)
    - Median distance (z-score)
    - Odd/even balance
    """
    feats = line_to_features(line_numbers)
    counts = feats["bucket_counts"].astype(float)

    # --- Bucket count score (Euclidean distance vs mean, normalized) ---
    mean_counts = hist_profile["mean_bucket_counts"]
    std_counts = hist_profile["std_bucket_counts"]
    # Avoid divide-by-zero
    std_counts_safe = np.where(std_counts == 0, 1.0, std_counts)

    z_counts = (counts - mean_counts) / std_counts_safe
    dist_counts = np.linalg.norm(z_counts)  # higher = further from typical

    # Convert distance to score ~ exp(-dist)
    bucket_score = np.exp(-dist_counts)

    # --- Bucket energy score ---
    if hist_profile["mean_bucket_energy"] is not None and hist_profile["std_bucket_energy"] not in (None, 0):
        z_energy = (feats["bucket_energy"] - hist_profile["mean_bucket_energy"]) / hist_profile["std_bucket_energy"]
        energy_score = np.exp(-abs(z_energy))
    else:
        energy_score = 1.0

    # --- Median score ---
    if hist_profile["mean_draw_median"] is not None and hist_profile["std_draw_median"] not in (None, 0):
        z_median = (feats["draw_median"] - hist_profile["mean_draw_median"]) / hist_profile["std_draw_median"]
        median_score = np.exp(-abs(z_median))
    else:
        median_score = 1.0

    # --- Odd/Even score ---
    if hist_profile["mean_odd"] is not None and hist_profile["mean_even"] is not None:
        odd_diff = abs(feats["Odd"] - hist_profile["mean_odd"])
        even_diff = abs(feats["Even"] - hist_profile["mean_even"])
        # Assume typical odd/even deviations of ~1.5 are fine
        odd_even_score = np.exp(-(odd_diff + even_diff) / 3.0)
    else:
        odd_even_score = 1.0

    # Combine scores (geometric mean for balance)
    scores = np.array([bucket_score, energy_score, median_score, odd_even_score])
    pattern_score = float(np.prod(scores) ** (1.0 / len(scores)))
    return pattern_score, {
        "bucket_score": float(bucket_score),
        "energy_score": float(energy_score),
        "median_score": float(median_score),
        "odd_even_score": float(odd_even_score),
    }


# Precompute historical profile once, if not already done
historical_profile = compute_historical_profiles(df_real_feat, bucket_count_cols)
print("Historical pattern profile computed.")


In [ ]:

# --- Extra visualisations ---

# 1) Real vs synthetic average bucket counts
if "df_synth_feat" in globals():
    real_avg_counts = df_real_feat[bucket_count_cols].mean()
    synth_avg_counts = df_synth_feat[bucket_count_cols].mean()

    x = np.arange(len(bucket_count_cols))
    width = 0.35

    plt.figure(figsize=(10, 4))
    plt.bar(x - width/2, real_avg_counts.values, width, label="Real")
    plt.bar(x + width/2, synth_avg_counts.values, width, label="Synthetic MC")
    plt.xticks(x, bucket_count_cols, rotation=45)
    plt.ylabel("Avg count per draw")
    plt.title("Average Bucket Counts: Real vs Monte Carlo Synthetic")
    plt.legend()
    plt.tight_layout()
    plt.show()


# 2) Bucket energy vs median: real vs synthetic
plt.figure(figsize=(6, 5))
if "draw_median" in df_real_feat.columns:
    plt.scatter(df_real_feat["draw_median"], df_real_feat["bucket_energy"], alpha=0.3, label="Real")
    if "df_synth_feat" in globals() and "draw_median" in df_synth_feat.columns:
        plt.scatter(df_synth_feat["draw_median"], df_synth_feat["bucket_energy"], alpha=0.2, label="Synthetic", marker="x")
    plt.xlabel("Draw Median")
    plt.ylabel("Bucket Energy")
    plt.title("Bucket Energy vs Draw Median (Real vs Synthetic)")
    plt.legend()
    plt.tight_layout()
    plt.show()


# 3) Example: score and visualise the last N generated prediction lines
example_lines = [
    np.array([3, 8, 16, 20, 23, 28]),
    np.array([3, 12, 18, 26, 29, 33]),
    np.array([2, 4, 8, 13, 18, 33]),
    np.array([8, 16, 19, 23, 33, 38]),
    np.array([3, 6, 9, 19, 37, 39]),
]

scores = []
for line in example_lines:
    s, breakdown = pattern_resonance_score(line, historical_profile, bucket_count_cols)
    scores.append((line, s, breakdown))

print("Example pattern resonance scores:")
for line, s, breakdown in scores:
    print(f"Line {line.tolist()} -> score={s:.3f}, components={breakdown}")

# Bar plot of overall scores
plt.figure(figsize=(8, 4))
labels = [f"L{i+1}" for i in range(len(scores))]
vals = [s for (_, s, _) in scores]
plt.bar(labels, vals)
plt.ylim(0, 1.0)
plt.ylabel("Pattern Resonance Score")
plt.title("Pattern Resonance Scores for Example Lines")
plt.tight_layout()
plt.show()


## Bulk Generation & Top-N Pattern-Resonant Predictions

This section:
- Generates a large number of model-based Lotto lines (e.g. 10,000)
- Computes the pattern resonance score for each line
- Sorts them and shows the top-N most Lotto-like lines according to historical structure.


In [ ]:
import numpy as np
import pandas as pd

def generate_model_line(feature_cols, use_model_mc=True):
    """Generate a single Lotto line using the current model and sanitiser."""
    candidate_numbers = np.sort(np.random.choice(np.arange(1, 41), size=6, replace=False))
    input_df = build_features_from_draw(candidate_numbers, feature_cols)

    if use_model_mc and "model_mc" in globals():
        pred = model_mc.predict(input_df.values)[0]
    else:
        pred = best_model.predict(input_df.values)[0]

    line = sanitize_prediction(pred, n_numbers=6, low=1, high=40)
    return line


def bulk_generate_and_score(
    n_samples=10000,
    top_n=20,
    use_model_mc=True,
    verbose_every=1000
):
    """Generate many model-based Lotto lines, score them, and return Top-N
    UNIQUE lines sorted by pattern resonance and frequency.
    """
    records = []

    for i in range(n_samples):
        # Generate one Lotto line using the ML model
        line = generate_model_line(feature_cols, use_model_mc=use_model_mc)

        # Score pattern resonance
        score, breakdown = pattern_resonance_score(line, historical_profile, bucket_count_cols)

        records.append({
            "line": line,
            "score": score,
            "bucket_score": breakdown["bucket_score"],
            "energy_score": breakdown["energy_score"],
            "median_score": breakdown["median_score"],
            "odd_even_score": breakdown["odd_even_score"],
        })

        if verbose_every and (i + 1) % verbose_every == 0:
            print(f"Generated and scored {i+1} lines...")

    df_scores = pd.DataFrame(records)

    # Make a dedupe key: "3,8,16,20,23,28"
    df_scores["line_str"] = df_scores["line"].apply(lambda arr: ",".join(map(str, arr)))

    # Aggregate unique line stats
    grouped = (
        df_scores
        .groupby("line_str")
        .agg(
            line=("line", "first"),
            score=("score", "max"),
            bucket_score=("bucket_score", "max"),
            energy_score=("energy_score", "max"),
            median_score=("median_score", "max"),
            odd_even_score=("odd_even_score", "max"),
            frequency=("line_str", "count"),
        )
        .reset_index(drop=True)
    )

    # Sort by score desc, then frequency desc
    df_sorted = grouped.sort_values(
        by=["score", "frequency"],
        ascending=[False, False]
    ).reset_index(drop=True)

    print(f"\nTop {top_n} UNIQUE pattern-resonant lines out of {n_samples} samples:\n")

    for idx in range(min(top_n, len(df_sorted))):
        row = df_sorted.iloc[idx]
        print(
            f"Rank {idx+1}: line={row['line'].tolist()}  "
            f"score={row['score']:.3f}  freq={row['frequency']}  "
            f"(bucket={row['bucket_score']:.3f}, "
            f"energy={row['energy_score']:.3f}, "
            f"median={row['median_score']:.3f}, "
            f"odd_even={row['odd_even_score']:.3f})"
        )

    return df_sorted

In [ ]:
df_topk = bulk_generate_and_score(
    n_samples=5000,
    top_n=20,
    use_model_mc=True,
    verbose_every=1000
)

## Extra Analysis of Top Pattern-Resonant Lines

These plots help understand how the top-ranked lines relate to historical structure:
- Histogram of resonance scores for the unique top lines
- Resonance vs line median
- Resonance vs bucket energy


In [ ]:

import matplotlib.pyplot as plt
import numpy as np

# Ensure df_topk exists
if 'df_topk' in globals() and not df_topk.empty:
    # 1) Histogram of resonance scores
    plt.figure(figsize=(8, 4))
    plt.hist(df_topk["score"], bins=15)
    plt.xlabel("Pattern Resonance Score")
    plt.ylabel("Frequency (unique lines)")
    plt.title("Distribution of Pattern Resonance Scores (Top Unique Lines)")
    plt.tight_layout()
    plt.show()

    # 2) Score vs median for each top line
    medians = []
    scores = df_topk["score"].values

    for line in df_topk["line"]:
        line_arr = np.array(line, dtype=int)
        medians.append(np.median(line_arr))

    medians = np.array(medians)

    plt.figure(figsize=(6, 4))
    plt.scatter(medians, scores)
    plt.xlabel("Line Median")
    plt.ylabel("Pattern Resonance Score")
    plt.title("Resonance Score vs Median of Top Lines")
    plt.tight_layout()
    plt.show()

    # 3) Score vs bucket energy for each top line
    energies = []
    for line in df_topk["line"]:
        feats = line_to_features(line)
        energies.append(feats["bucket_energy"])

    energies = np.array(energies)

    plt.figure(figsize=(6, 4))
    plt.scatter(energies, scores)
    plt.xlabel("Bucket Energy")
    plt.ylabel("Pattern Resonance Score")
    plt.title("Resonance Score vs Bucket Energy (Top Lines)")
    plt.tight_layout()
    plt.show()
else:
    print("df_topk is not defined or is empty. Run bulk_generate_and_score first.")
